In [1]:
#### 方便探索使用
import re
from sklearn.model_selection import KFold
from lightgbm import log_evaluation, early_stopping
import lightgbm as lgb
import os
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")



In [2]:
def del_PART_ORDER(PART_ORDER):
    """
    读取处理数据
    :return:
    """

    part_order = pd.read_csv(PART_ORDER).fillna(0)


    part_order['MONTH'] = pd.to_datetime(part_order['MONTH']).dt.strftime('%Y-%m-%d')
    part_order['MONTH_INT'] = part_order.apply(month_int_time, axis=1)
    # 去重
    part_order = part_order.drop_duplicates(subset=['PART_NO', 'MONTH_INT'], keep='last')
    # 排序
    part_order = part_order.sort_values(by='MONTH_INT')

    # print(part_order)

    return part_order


def del_PART_LIST(PART_LIST):

    part_list = pd.read_csv(PART_LIST, encoding="GBK")
    part_list['PIDT'] = pd.to_datetime(part_list['PIDT']).dt.strftime('%Y-%m-%d')
    ONEHOT_COLUMNS = ['CAR_CLASS', 'REPAIR_TYPE', 'TYPE_CODE', 'CONSTRUCT_NAME']

    DROP = ['KPDS', 'PIDT']
    for column in ONEHOT_COLUMNS:
        part_list = hot_coding_dispose(part_list, column)

    part_list = part_list.drop(columns=DROP)
    # 去重
    part_list = part_list.drop_duplicates(['PART_NO'],keep='last')

    return part_list

def del_TESTDATA_ID(TESTDATA_ID):
    part_predict_list = []
    with open(TESTDATA_ID, 'r') as f:
        for part in f.readlines():
            part_predict_list.append(part[:-1])

    return part_predict_list




def month_int_time(a):
    """
    从2015-1-1开始当前为第sum个月
    :param a:
    :return:
    """
    month = int(a['MONTH'].split('-')[1]) - 1
    year = (int(a['MONTH'].split('-')[0]) - 2015 ) * 12
    sum = year + month
    return sum

# def hot_encode():


def hot_coding_dispose(data, column):
    """
    使用热编码完成字符串转化
    :param data: 数据集
    :param column: 需要编码和删除的列名
    :return: 合并之后的集合
    """
    # 使用热门编码转化字符串
    code = pd.DataFrame({column: data[column]})
    code_DataFrame = pd.get_dummies(code)
    # merge，改用concat
    data = pd.concat([data, code_DataFrame], axis=1)
    # 删除列
    data = data.drop(columns=column)

    return data


def del_data(part_order, part_list, part_predict_list):
    """
    特征处理
    :param part_order:
    :param part_list:
    :param part_predict_list:
    :return:
    """
    part_train_x={}
    part_train_y1={}
    part_train_y3={}
    # 预测使用
    part_predict_x_lst={}
    part_len = len(part_predict_list)
    FEATURES = ['DM01','REPAIR_CNT','REPAIR_AMOUNT','RUN_DIST','MONTH_INT']
    break_time = 11

    # 合并特征
    part_order_list = pd.merge(part_order, part_list, how='left', left_on='PART_NO', right_on='PART_NO')
    del_feature = ['PART_NO', 'MONTH']
    features_all = [i for i in part_order_list.columns if i not in del_feature]

    for j in range(part_len):
        if (j % 100 == 0):
            print('\r', "数据处理进度：{}%".format(j*100/part_len), end="", flush=True)
        order_part = part_order_list[part_order_list['PART_NO'] == part_predict_list[j]]  # 依次取出零件历史销售数据
        for i in range(break_time, 81, 10):  ### （23,33,43,53,63，73）
            if i == break_time:
                ### 取i-23<=X<=i的内容
                train_x = order_part[(order_part['MONTH_INT'] <= i) & (order_part['MONTH_INT'] >= i - break_time)][FEATURES]
                ### train_x变为numpy并转置，reshape(1,-1)转化为一行
                train_x = train_x.to_numpy().T.reshape(1, -1)
                ### 获取第i+1个订单值 （未来一个月）
                train_y1 = order_part[(order_part['MONTH_INT'] == i + 1)]['DM01']
                ### 获取i+1,i+2,i+3的订单值的和 （未来三个月）
                train_y3 = np.array([order_part[(order_part['MONTH_INT'] == i + 1) | (order_part['MONTH_INT'] == i + 2) | (order_part['MONTH_INT'] == i + 3)]['DM01'].sum()])
            else:
                ### 取i-23<=X<=i的内容
                train_x_tmp = order_part[(order_part['MONTH_INT'] <= i) & (order_part['MONTH_INT'] >= i - break_time)][FEATURES]
                train_x_tmp = train_x_tmp.to_numpy().T.reshape(1, -1)
                train_y1_tmp = order_part[(order_part['MONTH_INT'] == i + 1)]['DM01']
                train_y3_tmp = np.array([order_part[((order_part['MONTH_INT'] == i + 1) | (order_part['MONTH_INT'] == i + 2) | (order_part['MONTH_INT'] == i + 3))]['DM01'].sum()])
                # 将某个零件的数据合并在一起
                train_x = np.append(train_x, train_x_tmp, axis=0)
                train_y1 = np.append(train_y1, train_y1_tmp, axis=0)
                train_y3 = np.append(train_y3, train_y3_tmp, axis=0)

        # 将所有零件的数据合并在一起
        part_train_x[part_predict_list[j]] = train_x
        part_train_y1[part_predict_list[j]] = train_y1  ###未来一个月
        part_train_y3[part_predict_list[j]] = train_y3  ###未来三个月

        #    最后一个输出数据
        i = 83
        train_x_lst = order_part[(order_part['MONTH_INT'] <= i) & (order_part['MONTH_INT'] >= i - break_time)][FEATURES]
        train_x_lst = train_x_lst.to_numpy().T.reshape(1, -1)
        part_predict_x_lst[part_predict_list[j]] = train_x_lst

    # 模型所用数据

    for part in part_predict_list:
        if part == part_predict_list[0]:
            train_x = part_train_x[part]
            train_y1 = part_train_y1[part]
            train_y3 = part_train_y3[part]
            predict_x_lst = part_predict_x_lst[part]
        else:
            train_x = np.concatenate((train_x, part_train_x[part]))
            train_y1 = np.concatenate((train_y1, part_train_y1[part]))
            train_y3 = np.concatenate((train_y3, part_train_y3[part]))
            predict_x_lst = np.concatenate((predict_x_lst, part_predict_x_lst[part]))

    return train_x, train_y1, train_y3, predict_x_lst, features_all

def format_model_data(part_train_x, part_predict_x_lst):

    return pd.DataFrame(part_train_x), pd.DataFrame(part_predict_x_lst)


In [3]:
PART_ORDER = "resources\PART_ORDER.csv"
PART_LIST = "resources\PART_LIST.csv"
TESTDATA_ID = "resources/testdata_id.txt"


part_order = del_PART_ORDER(PART_ORDER)
part_list = del_PART_LIST(PART_LIST)
part_predict_list = del_TESTDATA_ID(TESTDATA_ID)

part_train_x, part_train_y1, part_train_y3, part_predict_x_lst, features_all = del_data(part_order, part_list, part_predict_list)

part_train_x, part_predict_x_lst = format_model_data(part_train_x, part_predict_x_lst)


 数据处理进度：97.64758100310696%%

In [26]:
def mape_evaluate(preds, train_data):
    """
    评估方法
    :param preds:
    :param train_data:
    :return:
    """
    mape = 0.0
    train_y = train_data.get_label()
    for i in range(0, len(preds)):
        tmp = abs(float(int(train_y[i]) - int(preds[i]))) / (max(float(train_y[i]), float(preds[i])) + 1)
        mape += tmp
    mape /= len(preds)
    return "mape", 1-mape


In [23]:
def train_model(train_x, train_y, learning_rate, num_boost_round, stopping_rounds, objective, boosting):

    params = {'num_leaves': 60,
              'min_data_in_leaf': 50,
              'objective': objective,
              'max_depth': -1,
              'learning_rate': learning_rate,
              "min_sum_hessian_in_leaf": 6,
              "boosting": boosting,
              "feature_fraction": 0.9,
              "bagging_freq": 1,
              "bagging_fraction": 0.7,
              "bagging_seed": 11,
              "lambda_l1": 0.1,
              # 'lambda_l2': 0.001,
              "verbosity": -1,
              "nthread": 3,
              'metric': None,
              "random_state": 111,
              }
    folds = KFold(n_splits=5, shuffle=True, random_state=111)
    score = 0.0
    for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_x)):
        print("fold {}".format(fold_ + 1))
        trn_data = lgb.Dataset(train_x.iloc[trn_idx], label=train_y[trn_idx])
        val_data = lgb.Dataset(train_x.iloc[val_idx], label=train_y[val_idx])
        clf = lgb.train(params,
                trn_data,
                num_boost_round=num_boost_round,
                valid_sets=[trn_data, val_data],
                valid_names=['train','train_test'],
                callbacks=[log_evaluation(period=False),
                           early_stopping(stopping_rounds=stopping_rounds)],
                feval=mape_evaluate
                )

        score += clf.best_score['train_test']['mape']
    return score / folds.n_splits


In [27]:
import xgboost as xgb
train_x = part_train_x

train_y1 = part_train_y1
train_y3 = part_train_y3
train_y = part_train_y1
params = {
    'booster': 'gbtree',
    'min_child_weight': 10.0,
    'learning_rate': 0.02,
    'objective': 'reg:squarederror',
    # 'eval_metric': "mape",
    'max_depth': 7,
    'max_delta_step': 1.8,
    'colsample_bytree': 0.4,
    'subsample': 0.8,
    'eta': 0.025,
    'gamma': 0.65,
    'nthread': -1,
    'seed': 111,
}



folds = KFold(n_splits=2, shuffle=True, random_state=111)
score = 0.0
for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_x)):
    print("fold {}".format(fold_ + 1))
    trn_data = xgb.DMatrix(train_x.iloc[trn_idx], label=train_y[trn_idx])
    val_data = xgb.DMatrix(train_x.iloc[val_idx], label=train_y[val_idx])

    watchlist = [(trn_data, 'train'), (val_data, 'valid')]
    clf = xgb.train(params,
                    trn_data,
                    num_boost_round=1000,
                    evals=watchlist,
                    custom_metric=mape_evaluate,
                    verbose_eval=200,)

    print(clf.best_iteration)
print(score)
    # test_pred_prob += clf.predict(xgb.DMatrix(test), ntree_limit=clf.best_iteration) / folds.n_splits

fold 1
[0]	train-rmse:761.30919	train-mape:0.27051	valid-rmse:649.42408	valid-mape:0.25584
[200]	train-rmse:760.19244	train-mape:0.45189	valid-rmse:648.15395	valid-mape:0.43773
[400]	train-rmse:759.12375	train-mape:0.55770	valid-rmse:646.94110	valid-mape:0.54023
[600]	train-rmse:758.09618	train-mape:0.62823	valid-rmse:645.77769	valid-mape:0.60804
[800]	train-rmse:757.10130	train-mape:0.66784	valid-rmse:644.65401	valid-mape:0.64621
[999]	train-rmse:756.13872	train-mape:0.69494	valid-rmse:643.56923	valid-mape:0.67102
999
fold 2
[0]	train-rmse:649.42408	train-mape:0.25584	valid-rmse:761.30919	valid-mape:0.27051
[200]	train-rmse:648.15365	train-mape:0.44144	valid-rmse:760.19280	valid-mape:0.45233
[400]	train-rmse:646.93933	train-mape:0.55222	valid-rmse:759.12463	valid-mape:0.56168
[600]	train-rmse:645.77352	train-mape:0.62515	valid-rmse:758.09934	valid-mape:0.62740
[800]	train-rmse:644.64726	train-mape:0.66439	valid-rmse:757.10900	valid-mape:0.65834
[999]	train-rmse:643.55962	train-mape:0.